In [173]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [213]:
df1 = pd.read_csv('magentacloud/output_Marktplatz-20180410-141306.csv')
df1 = df[df['w']<100] # filter large boxes

In [214]:
df1['distance'] = df.groupby(['track_id']).diff().dropna().apply(
    lambda row : np.sqrt((row['x']*.026)**2 + (row['y']*.037)**2), axis = 1)
df1['frame_diff'] = df.groupby(['track_id'])[['frame_id']].diff()

In [215]:
df1['velocity'] = df['distance'] / ( df['frame_diff'] * (1.0/30))
df1['velocity_smooth'] = df.groupby(['track_id'])['velocity'].apply(pd.rolling_mean, 5, min_periods=1)

/home/florian/anaconda3/lib/python3.6/site-packages/pandas/core/groupby.py:796: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=5,min_periods=1,center=False).mean()
  return func(g, *args, **kwargs)


In [216]:
counts = df1.groupby(['frame_id']).count().reset_index()[['frame_id', 'track_id']].rename({'track_id':'count'},axis=1)

In [217]:
df2 = pd.merge(df1,counts,on='frame_id')

In [226]:
df2.sort_values(['frame_id','velocity_smooth'], ascending=False).drop_duplicates(['frame_id']).sort_values('frame_id')

,frame_id,track_id,x,y,w,h,distance,frame_diff,velocity,velocity_smooth,count
0,12,1,1055,337,40,82,NaN,NaN,NaN,NaN,4
4,16,1,1039,335,39,78,NaN,4.0,3.168979,3.168979,4
8,20,1,1029,334,36,71,0.262619,4.0,1.969646,2.569312,6
18,24,11,1504,552,56,93,NaN,4.0,5.692124,5.692124,6
24,28,11,1513,562,47,77,0.437785,4.0,3.283390,4.487757,6
30,32,11,1517,561,47,78,0.110386,4.0,0.827893,3.267802,7
37,36,11,1524,565,45,77,0.234580,4.0,1.759354,2.890690,7
44,40,11,1529,569,43,72,0.196987,4.0,1.477405,2.608033,7
51,44,11,1539,575,42,71,0.341883,4.0,2.564123,1.982433,8
58,48,11,1549,574,44,77,0.262619,4.0,1.969646,1.719684,7


In [237]:
json = df2[['frame_id', 'track_id', 'velocity_smooth', 'count']].to_json(orient='records')

In [240]:
with open("json.txt", "w") as text_file:
    text_file.write(json)